### Data Preprocessing Pipeline for Image Segmentation
- We are developing a simple data preprocessing pipeline using OpenCV and PyTorch for image segmentation tasks.
- This pipeline includes loading an image and its corresponding mask, resizing, normalization, augmentation (random horizontal flip), and converting to PyTorch tensors.

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


#### 1. Import libraries

In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
import os # Import the os module

#### 2. Load image and mask

In [ ]:
def load_image_and_mask(image_path, mask_path):
    # Check if files exist
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        return None, None
    if not os.path.exists(mask_path):
        print(f"Error: Mask file not found at {mask_path}")
        return None, None

    # Load image in BGR, convert to RGB
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image from {image_path}. It might be corrupted or an unsupported format.")
        return None, None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load mask in grayscale (assumed: 0 = background, >0 = subject)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Error: Could not read mask from {mask_path}. It might be corrupted or an unsupported format.")
        return None, None

    return image, mask

# Example usage
image_path = "/content/drive/MyDrive/photos/IMG_20231225_170619_1 (1).jpg"
mask_path  = "/content/drive/MyDrive/photos/IMG_20231225_170702 (1).jpg"

image, mask = load_image_and_mask(image_path, mask_path)

if image is not None and mask is not None:
    print(f"Image shape: {image.shape}, Mask shape: {mask.shape}")
else:
    print("Image or mask failed to load. Please check the error messages above.")

error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


- `cv2.imread(image_path)` loads the image in BGR format (OpenCV default).

- `cv2.cvtColor(..., cv2.COLOR_BGR2RGB)` converts it to RGB so it matches the usual PyTorch convention.

- `cv2.imread(..., cv2.IMREAD_GRAYSCALE)` loads the mask as a single-channel grayscale image (0 for background, non-zero for subject).

#### 3. Resize image and mask

In [ ]:
def resize_image_and_mask(image, mask, target_size=(256, 256)):
    # Resize image using bilinear interpolation
    image_resized = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

    # Resize mask using nearest neighbor (to avoid blurring class boundaries)
    mask_resized = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)

    return image_resized, mask_resized

# Example usage
target_size = (256, 256)
image_resized, mask_resized = resize_image_and_mask(image, mask, target_size)
print(f"Resized image shape: {image_resized.shape}, mask shape: {mask_resized.shape}")

- `cv2.INTER_LINEAR` is good for images (smooth resizing).

- `cv2.INTER_NEAREST` is used for masks to preserve sharp boundaries (no interpolation between classes).

- `target_size` is usually a tuple like `(H, W)` or `(W, H)`; OpenCV uses `(width, height)`.

#### 4. Normalize image to and convert to float

In [ ]:
def normalize_image(image):
    # Convert to float32 and scale to [0, 1]
    image_float = image.astype(np.float32) / 255.0
    return image_float

# Example usage
image_normalized = normalize_image(image_resized)
print(f"Image dtype: {image_normalized.dtype}, range: [{image_normalized.min():.3f}, {image_normalized.max():.3f}]")

- Converts the image from uint8 (0–255) to float32 (0.0–1.0).

- This is needed before applying PyTorch normalization transforms.

#### 5. Apply random horizontal flip (augmentation)

In [ ]:
def random_horizontal_flip(image, mask, p=0.5):
    if np.random.rand() < p:
        image = np.flip(image, axis=1)
        mask  = np.flip(mask,  axis=1)
    return image, mask

# Example usage
image_aug, mask_aug = random_horizontal_flip(image_normalized, mask_resized, p=0.5)

- Flips both image and mask horizontally with probability p.

- This is a simple data augmentation that helps the model generalize better.

- Always flip the mask in the same way as the image so the labels stay aligned.

#### 6. Convert to PyTorch tensors

In [ ]:
def to_tensor(image, mask):
    # Handle negative strides by making a copy if necessary (common after np.flip)
    if image.strides[0] < 0 or image.strides[1] < 0:
        image = image.copy()
    if mask.strides[0] < 0 or mask.strides[1] < 0:
        mask = mask.copy()

    # Convert image: HWC → CHW and to tensor
    image_tensor = torch.from_numpy(image).permute(2, 0, 1)  # HWC → CHW
    mask_tensor  = torch.from_numpy(mask).long()              # mask as long tensor

    return image_tensor, mask_tensor

# Example usage
image_tensor, mask_tensor = to_tensor(image_aug, mask_aug)
print(f"Image tensor shape: {image_tensor.shape}, mask tensor shape: {mask_tensor.shape}")

- `permute(2, 0, 1)` changes the order from `(H, W, C)` to `(C, H, W)` as expected by PyTorch models.

- `mask` is converted to `long` (int64) because segmentation masks are class indices (not floats).

#### 7. Normalize image using ImageNet stats

In [ ]:
# Define normalization transform (ImageNet mean/std)
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

# Apply normalization
image_normalized_tensor = normalize(image_tensor)

- Normalizes each channel using ImageNet statistics (common for models like ResNet, VGG, etc.).

- This helps the model train faster and more stably.

- Only applied to the image; the mask is left as-is.

#### 8. Convert mask to binary (subject vs background)

In [ ]:
def make_binary_mask(mask_tensor):
    # Convert any non-zero value to 1 (subject), 0 remains background
    binary_mask = (mask_tensor > 0).long()
    return binary_mask

# Example usage
binary_mask = make_binary_mask(mask_tensor)
print(f"Unique values in mask: {torch.unique(binary_mask)}")  # Should be tensor([0, 1])

- Converts a multi-class mask into a binary mask:

  - `0` → background

  - `>0` → subject (set to 1)

- Useful if the original mask has multiple object classes but the task is just “subject vs background”.


#### Task: Implement the full pipeline and execute it.

In [ ]:
def preprocess_pipeline(image_path, mask_path, target_size=(256, 256), p_flip=0.5):
    # 1. Load image and mask
    image, mask = load_image_and_mask(image_path, mask_path)

    # 2. Resize image and mask
    image_resized, mask_resized = resize_image_and_mask(image, mask, target_size)

    # 3. Normalize image to [0, 1] float
    image_normalized_float = normalize_image(image_resized)

    # 4. Apply random horizontal flip
    image_augmented, mask_augmented = random_horizontal_flip(image_normalized_float, mask_resized, p=p_flip)

    # 5. Convert to PyTorch tensors
    image_tensor, mask_tensor = to_tensor(image_augmented, mask_augmented)

    # 6. Apply ImageNet normalization (PyTorch transform)
    image_normalized_final = normalize(image_tensor)

    # 7. Convert mask to binary
    binary_mask_final = make_binary_mask(mask_tensor)

    return image_normalized_final, binary_mask_final


processed_image, processed_mask = preprocess_pipeline(image_path, mask_path, target_size=(256, 256), p_flip=0.5)

print(f"Final Processed Image Tensor Shape: {processed_image.shape}")
print(f"Final Processed Mask Tensor Shape: {processed_mask.shape}")
print(f"Unique values in final mask: {torch.unique(processed_mask)}")